In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
# from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
from category_encoders import *
from sklearn.impute import KNNImputer
from sklearn import decomposition as dp
from sklearn.metrics import confusion_matrix, f1_score,classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit


In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# print(test)

def count_class(rep):
    target_count = rep.value_counts()
    print(target_count)
    print()
    print('Percentge of target ', np.round(100 * target_count[0] / target_count.sum(), 2), '%')
    
    
    
count_class(train['Response'])

train

number_of_negative_samples = 1329
number_of_positive_samples = 239

0    1329
1     239
Name: Response, dtype: int64

Percentge of target  84.76 %


In [3]:
test['Response'] = 3

data = pd.concat([train,test])

data = data.drop(['ID'],1)



In [4]:
def cat_count(L_cat):
    for i in L_cat.columns.tolist():
        print(L_cat[i].value_counts())
        print(  )
        print('the length is:',len(L_cat[i].unique()))
        print('------------')
        print('------------')
        
        
        
cat_count(data)

1977    89
1972    87
1976    83
1973    79
1971    77
1979    77
1966    74
1974    74
1970    71
1975    69
1957    55
1959    53
1980    53
1978    52
1953    52
1969    51
1960    51
1967    50
1955    50
1956    49
1961    49
1983    45
1964    45
1963    44
1968    44
1958    43
1952    43
1965    42
1987    42
1984    42
1982    39
1981    39
1985    38
1962    36
1954    35
1986    32
1950    30
1990    30
1951    29
1989    29
1988    27
1949    21
1991    18
1948    16
1947    16
1992    15
1993    13
1946     8
1945     7
1944     7
1994     5
1996     5
1995     3
1997     2
1901     1
1941     1
1942     1
1900     1
1894     1
Name: Year_of_Birth, dtype: int64

the length is: 59
------------
------------
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: Education_Level, dtype: int64

the length is: 5
------------
------------
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
A

In [5]:
impute = KNNImputer(n_neighbors= 2)
data['Disposable_Income']=impute.fit_transform(data[['Disposable_Income']])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 671
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year_of_Birth            2240 non-null   int64  
 1   Education_Level          2240 non-null   object 
 2   Marital_Status           2240 non-null   object 
 3   Disposable_Income        2240 non-null   float64
 4   No_of_Kids_in_home       2240 non-null   int64  
 5   No_of_Teen_in_home       2240 non-null   int64  
 6   Date_Customer            2240 non-null   object 
 7   Recency                  2240 non-null   int64  
 8   Discounted_Purchases     2240 non-null   int64  
 9   WebPurchases             2240 non-null   int64  
 10  CatalogPurchases         2240 non-null   int64  
 11  StorePurchases           2240 non-null   int64  
 12  Amount_on_Wines          2240 non-null   int64  
 13  Amount_on_Fruits         2240 non-null   int64  
 14  Amount_on_MeatProducts   

In [7]:
data['Date_Customer'] = pd.to_datetime(data.Date_Customer )
data['Date_Customer']

0     2014-06-22
1     2013-01-08
2     2013-11-21
3     2013-09-13
4     2014-01-21
         ...    
667   2012-12-08
668   2013-02-18
669   2013-09-08
670   2013-08-19
671   2013-11-13
Name: Date_Customer, Length: 2240, dtype: datetime64[ns]

In [8]:
# data['purchase_week'] = data[['Date_Customer']].applymap(lambda dt:dt.week
#                                                 if not pd.isnull(dt.week)
#                                                 else 0)
# data['purchase_month'] = data[['Date_Customer']].applymap(lambda dt:dt.month
#                                                 if not pd.isnull(dt.month)
#                                                 else 0)

# data['purchase_day'] = data[['Date_Customer']].applymap(lambda dt:dt.day
#                                                 if not pd.isnull(dt.day)
#                                                 else 0)

data['purchase_year'] = data[['Date_Customer']].applymap(lambda dt:dt.year
                                                if not pd.isnull(dt.year)
                                                else 0)

# data['purchase_year'] = 2020 - data['purchase_year'] 

# data['Year_of_Birth'] = 2020 - data['Year_of_Birth']

# data['total_children'] = data['No_of_Kids_in_home'] + data['No_of_Teen_in_home']

# data['Disposable_Income_log'] = np.log((1+ data['Disposable_Income']))

In [9]:
data = data.drop(['Date_Customer'],1)
# data = data.drop(['No_of_Kids_in_home','No_of_Teen_in_home'],1)
# data = data.drop(['Disposable_Income'],1)

In [10]:
data


train_T =data[data['Response'] != 3]

Target_T = train_T['Response']

enc = TargetEncoder(cols=data[['Marital_Status','Education_Level']])

ordin = OrdinalEncoder(cols=data[['Education_Level']])


# ordin.fit(train_T, Target_T)

# data = ordin.transform(data) 


enc.fit(train_T, Target_T)

data = enc.transform(data) 





def drop_outliers(df,threshold):
    from scipy import stats
    z = stats.zscore(df)
    Num_a = df[(z<threshold).all(axis = 1)]
    print("shape is :",Num_a.shape)
    return Num_a



In [11]:
# import seaborn as sns


# corr = data.corr()

# sns.heatmap(corr)


# columns = np.full((corr.shape[0],), True, dtype=bool)
# for i in range(corr.shape[0]):
#     for j in range(i+1, corr.shape[0]):
#         if corr.iloc[i,j] >= 0.9:
#             if columns[j]:
#                 columns[j] = False
# selected_columns = data.columns[columns]
# data = data[selected_columns]





# selected_columns = selected_columns[1:].values
# import statsmodels.api as sm
# def backwardElimination(x, Y, sl, columns):
#     numVars = len(x[0])
#     for i in range(0, numVars):
#         regressor_OLS = sm.OLS(Y, x).fit()
#         maxVar = max(regressor_OLS.pvalues).astype(float)
#         if maxVar > sl:
#             for j in range(0, numVars - i):
#                 if (regressor_OLS.pvalues[j].astype(float) == maxVar):
#                     x = np.delete(x, j, 1)
#                     columns = np.delete(columns, j)
                    
#     regressor_OLS.summary()
#     return x, columns
# SL = 0.05
# data_modeled, selected_columns = backwardElimination(data.iloc[:,1:].values, data.iloc[:,0].values, SL, selected_columns)
    

    
    
# result = pd.DataFrame()
# result['diagnosis'] = data.iloc[:,0]

# data = pd.DataFrame(data = data_modeled, columns = selected_columns)

# data['Response'] = Target_T 

# data.shape


In [12]:
# data['Response'] = np.array(data['Response'])

# r_target= pd.to_numeric(data['subscribed'])






test2 =data[data['Response'] == 3]
test3 = test2.drop(['Response'],axis = 1)


# test = test.astype('int32')


print(test3.shape)

train_b =data[data['Response'] != 3]

# train_b= drop_outliers(train_b,3)

Target = train_b['Response']
N_dat = train_b.drop(['Response'],axis = 1)

N_dat







(672, 25)


,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Recency,Discounted_Purchases,WebPurchases,CatalogPurchases,...,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,purchase_year
0,1955,0.143392,0.201258,77504.4,1,1,56,2,1,1,...,6,20,3,0,0,0,0,0,0,2014
1,1958,0.143392,0.100244,56784.0,0,1,17,6,5,1,...,15,19,6,0,0,0,0,0,0,2013
2,1962,0.143392,0.215805,103714.8,0,0,17,0,7,4,...,18,37,2,0,0,0,1,0,0,2013
3,1979,0.098592,0.215805,46311.6,1,0,49,3,4,1,...,24,24,8,0,0,0,0,0,0,2013
4,1959,0.143392,0.126829,87486.0,0,0,59,1,3,7,...,165,203,1,0,0,0,1,0,0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,1970,0.143392,0.126829,52893.6,1,1,24,2,2,0,...,0,4,5,0,0,0,0,0,0,2014
1564,1980,0.143392,0.215805,74190.0,0,1,63,1,4,2,...,30,27,4,0,0,0,0,0,0,2013
1565,1952,0.154472,0.126829,80857.2,0,1,74,4,2,2,...,0,70,7,0,1,0,0,0,0,2013
1566,1964,0.208824,0.215805,58701.6,1,1,28,2,1,0,...,0,1,4,0,0,0,0,0,0,2014


In [13]:




test4 = test3




# test4

# ret

In [14]:

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error ,accuracy_score
import lightgbm as lgb



from imblearn.over_sampling import RandomOverSampler,SMOTE


smote = SMOTE()

# ros = RandomOverSampler(random_state = 100)
t_data, t_target = smote.fit_resample(N_dat,Target)


# t_data = N_dat
# t_target = Target

# test4= pd.DataFrame(test4,columns= test3.columns)



In [15]:



def kfold(x,y,trial):
    from sklearn.metrics import accuracy_score
    
    X = x
    y = y
    
    err=[]
    y_pred =[]
    
    fold=StratifiedKFold(n_splits=2, random_state=2020, shuffle=True)
    
    for train_index, test_index in fold.split(X,y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        dtrain = lgb.Dataset(X_train, label=y_train)
        
        param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        }

        gbm = lgb.train(param, dtrain)
        preds = gbm.predict(X_test)
        pred_labels = np.rint(preds)
        accuracy = accuracy_score(y_test, pred_labels)
        return accuracy






# def objective(trial):
#     accuracy = kfold(t_data, t_target,trial)
#     return accuracy



# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [16]:


# X = D_data




para = {
        "objective": "binary",
#         "metric": "binary_logloss",
        "metric": "f1_score",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": 0.0000024091426688585824,
        "lambda_l2": 0.00000001418801233105257,
        "num_leaves": 111,
        "feature_fraction": 0.45550312924217073,
        "bagging_fraction":  0.8102605560323656,
        "bagging_freq": 2,
        "min_child_samples": 8,
       
        }



gbm= lgb.LGBMClassifier(objective =  'binary',
#         "metric": "binary_logloss",
        metric= "f1_score",
        verbosity= -1,
        boosting_type= "gbdt",
        lambda_l1= 0.0000024091426688585824,
        lambda_l2= 0.00000001418801233105257,
        num_leaves= 111,
        feature_fraction= 0.45550312924217073,
        bagging_fraction=  0.8102605560323656,
        bagging_freq= 2,
        min_child_samples= 2,
                       )



X = t_data

y = t_target


err=[]
y_pred =[]

# fold=StratifiedKFold(n_splits=2, shuffle=True)


fold=StratifiedKFold(n_splits=2, random_state=2020, shuffle=True)

for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     dtrai = lgb.Dataset(X_train, label=y_train)
    
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    print(accuracy)
#     preds=model.predict(X_test)
#     print("accuracy: ",f1_score(y_test,preds))     
    err.append(f1_score(y_test,pred_labels))
    print

# print(accuracy)   
print(classification_report(y_test,pred_labels))
print(np.mean(err))







# def kfold(x,y):
    
#     X = x
#     y = y
    
#     err=[]
#     y_pred =[]
    
#     fold=StratifiedKFold(n_splits=2, random_state=2020, shuffle=True)
    
#     for train_index, test_index in fold.split(X,y):
#         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
#         dtrain = lgb.Dataset(X_train, label=y_train)
        
#         param = {
#         "objective": "binary",
#         "metric": "binary_logloss",
#         "verbosity": -1,
#         "boosting_type": "gbdt",
#         "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
#         "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 256),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#         }

#         gbm = lgb.train(param, dtrain)
#         preds = gbm.predict(X_test)
#         pred_labels = np.rint(preds)
#         accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
#         return accuracy
        
        
        
    
   

    




# model=CatBoostClassifier(iterations=40, random_seed=2020, eval_metric='AUC',early_stopping_rounds =20)
#     model=CatBoostClassifier(n_estimators = 5000,  random_seed=2020, eval_metric='AUC',early_stopping_rounds =20)    
#     model = LogisticRegression()
#     model = neighbors.KNeighborsClassifier(2)
# model= lgb.LGBMClassifier(max_depth = 8)

0.9367945823927766
0.9360421369450714
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       664
           1       0.94      0.93      0.94       665

    accuracy                           0.94      1329
   macro avg       0.94      0.94      0.94      1329
weighted avg       0.94      0.94      0.94      1329

0.9366185775752973


In [17]:
#submit=pd.read_csv("SampleSubmission.csv")
#submit["Response"]=gbm.predict(test4)
#print(submit)
#submit.to_csv("submission8.csv",index=False)

In [18]:
f1_score(y_test, pred_labels)

0.9354593773728169